In [ ]:
#https://arxiv.org/abs/1405.4053
#https://radimrehurek.com/gensim/models/doc2vec.html
#https://radimrehurek.com/gensim/auto_examples/tutorials/run_doc2vec_lee.html

In [ ]:
import pandas as pd

In [ ]:
df_train = pd.read_parquet('./data/movie-genre-prediction/train.parquet', engine='pyarrow')
df_test = pd.read_parquet('./data/movie-genre-prediction/test.parquet', engine='pyarrow')

In [ ]:
df_train.head()

In [ ]:
df_train.shape

In [ ]:
genres_list = list(df_train.genre.unique())
genres_list

In [ ]:
df_test.shape

In [ ]:
df_train["text"] = df_train.movie_name.str.lower() + ". " + df_train.synopsis.str.lower()
df_train.head()

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
print(word_tokenize(df_train.iloc[0,:]["text"]))
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
print(tokenizer.tokenize(df_train.iloc[0,:]["text"]))

In [ ]:
data = df_train.iloc[0:10,:]["text"] #small dataset
#data = df_train["text"] #full dataset
data[0]

In [ ]:
tagged_data = [TaggedDocument(words=word_tokenize(d.lower()), 
                              tags=[str(i)]) for i, d in enumerate(data)]

In [ ]:
tagged_data[0]

In [ ]:
model = Doc2Vec(vector_size=5,min_count=2,epochs=40,window=5)

In [ ]:
model.build_vocab(tagged_data)

In [ ]:
from gensim.models.callbacks import CallbackAny2Vec
class callback(CallbackAny2Vec):
    def __init__(self):
        self.epoch = 1
    def on_epoch_end(self, model):
        if self.epoch % 10 == 0:
            print(f"Epoch {self.epoch} finished")
        self.epoch += 1

In [ ]:
model.train(tagged_data,
            total_examples=model.corpus_count, 
            epochs=model.epochs,
            callbacks=[callback()])
print("Train finished")

In [ ]:
model.save("d2v_muestra.model")
print("Modelo Guardado")

In [ ]:
# Cree vectores de 5 dimensiones para las 10 primeras peliculas
for vid in range(len(model.dv)):
    print(model.dv[vid])

In [ ]:
len(model.dv)

In [ ]:
model.dv[0]

In [ ]:
# cargo el modelo
model= Doc2Vec.load("d2v.model")

# creo el vector para un documento nuevo no antes visto en el entrenamiento
test_data = word_tokenize("Este es un texto nuevo".lower())
v1 = model.infer_vector(test_data)
print("Vector inferido:", v1)

In [ ]:
df_test["text"] = df_test.movie_name.str.lower() + " " + df_test.synopsis.str.lower()
df_test.head()